In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
from datetime import datetime

In [2]:
uber_db=pd.read_csv(r'C:\Users\mahe\Desktop\Upgrad\EDA statistics and Visualization\Uber case study\Uber Request Data.csv')

In [5]:
#Check for datatpes
uber_db.dtypes

Request id             int64
Pickup point          object
Driver id            float64
Status                object
Request timestamp     object
Drop timestamp        object
dtype: object

In [ ]:
#Remove unwanted tabs and spaces in Request Time stamp and Drop Time stamp
uber_db['Request timestamp']=uber_db['Request timestamp'].astype(str).apply(lambda x: x.strip())
uber_db['Drop timestamp']=uber_db['Drop timestamp'].astype(str).apply(lambda x: x.strip())
uber_db.head(20)

In [ ]:
#Cleaning data using function cleantimestamp
uber_db['Request timestamp']=uber_db['Request timestamp'].astype(str).apply(lambda x: x.replace('/','-'))
import re
def clean_timestamp(timestamp):
        #print(timestamp)
        if timestamp == 'nan':
            return timestamp
        reg_search = re.search(r'''(?P<day>[\d]+)-(?P<month>[\d]+)-(?P<year>[\d]+) (?P<hour>[\d]+):(?P<mins>[\d]+)''',
                               timestamp)
        res = reg_search.groupdict()
        if len(res['month'])  == 1:
            res['month'] = '0{}'.format(res['month'])
        if len(res['day'])  == 1:
            res['day'] = '0{}'.format(res['day'])
        new_timestamp = '{}-{}-{} {}:{}'.format(res['day'], res['month'], res['year'], res['hour'], res['mins'])
        new_timestamp = datetime.strptime(new_timestamp,'%d-%m-%Y %H:%M')
        return new_timestamp

uber_db['Request timestamp']=uber_db['Request timestamp'].apply(lambda x : clean_timestamp(x))
uber_db.head(200)

In [ ]:
#Replacing / by - for standardising the data
uber_db['Drop timestamp']=uber_db['Drop timestamp'].astype(str).apply(lambda x: x.replace('/', '-'))
print('Updting the drop timestamp')
uber_db['Drop timestamp']=uber_db['Drop timestamp'].apply(lambda x : clean_timestamp(x))

uber_db.head(20)

In [ ]:
#after standardising the data we check for duplicates
#Check for any duplicates
uber_db[uber_db.duplicated()].shape

In [ ]:
#calculating journey time
uber_db['journey_time']=uber_db['Drop timestamp']-uber_db['Request timestamp']
uber_db['journey_hour']=uber_db['journey_time'].apply(lambda x : x.datatime.hours)
uber_db['journey_minute']=uber_db['journey_time'].apply(lambda x : x.datatime.minute)


In [ ]:
# Extracting Univariate data such as time and date
uber_db['request_hour']=uber_db['Request timestamp'].apply(lambda x:x.hour)
uber_db['drop_hour']=uber_db['Drop timestamp'].apply(lambda x:x.hour)

#Extracting Univariate for further analysis Request
uber_db['request_month']=uber_db['Request timestamp'].apply(lambda x: x.month)
uber_db['request_day']=uber_db['Request timestamp'].apply(lambda x: x.day)
uber_db['request_year']=uber_db['Request timestamp'].apply(lambda x: x.year)

#Extracting Univariate for further analysis Drop
uber_db['drop_month']=uber_db['Drop timestamp'].apply(lambda x: x.month)
uber_db['drop_day']=uber_db['Drop timestamp'].apply(lambda x: x.day)
uber_db['drop_year']=uber_db['Drop timestamp'].apply(lambda x: x.year)



In [ ]:
#Different datasets for different status 
#Trip Not Completed
uber_null=uber_db[uber_db['Drop timestamp'].astype(str) =='NaT']
uber_null.shape

#Trip Completed
uber_tripcompleted=uber_db[uber_db['Drop timestamp'].astype(str)!='NaT']
uber_tripcompleted.shape

#No cars Available
uber_nocars=uber_db[uber_db['Driver id'].isnull()==True]
uber_nocars.shape

In [ ]:
#Plotting data to find outliers
#plotting boxplot on journey time o find any outliers
import matplotlib.pyplot as plt
plt.boxplot(uber_db['journey_time'])
uber_db['journey_time'].describe()
#Scatter plot
plt.scatter(uber_db['request_day'],uber_db['drop_day'])

#Histogram on journey hours for calculating peak time in data
uber_tripcompleted['journey_hour']=uber_tripcompleted['journey_time'].apply(lambda x:x.datetime.hours)
#time series plot
import seaborn as sns
sns.tsplot(data=uber_tripcompleted['journey_hour'])

In [ ]:
#Checking the spread of the data by scatter plot
# to check any outlier that is, is there any data that shows trip time more than 24 hours
plt.plot(uber_db['request_day'],uber_db['drop_day'])
plt.xlabel("Request Day")
plt.ylabel("Drop Day")
plt.title("Cross checking Request vs Drop")

#Since its positive and all the variables are in boundries, there fore no outliers

In [ ]:
#Checking outliers by using different plots
#Inititiating new Figure
plt.figure(1)
#Creating sublots with 1 row 2 column on 1st
plt.subplot(1,2,1)
plt.title("Request timestamp")
plt.plot(uber_db['Request timestamp'],uber_db['Driver id'])

#Creating 2nd Graph
plt.subplot(1,2,2)
plt.title("Drop timestamp")
plt.plot(uber_db['Drop timestamp'],uber_db['Driver id'])

In [ ]:
#PLotting Univariate 
sns.distplot(uber_db['request_day'],bins=50)
#We can see the data distributed normally where as the count is maximum
sns.distplot(uber_tripcompleted['drop_day'],bins=50)
sns.show

#Bivariate
uber_db_temp=uber_db[(uber_db.journey_hour < 3) & (uber_db.status='Trip Completed')]
sns.jointplot(uber_db['journey_hour'],uber_tripcompleted['drop_hour'],uber_db_temp)
plt.show()
            

In [ ]:
#Category wise plotting, treating category as driver
#Seaborn
sns.set(style='white')
#Boxplot of var
sns.boxplot(y=uber_db['Driver id'])
plt.show
#Boxplot variable accross driver
sns.boxplot(x='Driver id',y='journey_time',data=uber_tripcompleted)
plt.show()

In [ ]:
#Time Series plot on drop and journey to find month wise highest 
#Initially we con verted the data to datetype
uber_db['drop_time']=pd.to_datetime(uber_db['drop_time'])
#aggregating total journey time day wise
journey_data=uber_db.groupby('drop_hour')['journey_time'].sum()
print(journey_data.head())

#Timeseries plot
sns.tsplot(data=journey_data)
plt.show()

#group by month and year to see the highest 
journey_data=uber_db.groupby(['drop_month','drop_year']).journey_time.mean()

#Combined timeseries data
plt.figure(figsize=(8,6))
sns.tsplot(journey_data)
plt.xlabel("Journey time")
plt.ylabel("Drop time")
plt.show()

In [ ]:
#Time Series plot on Request and journey to find month wise highest 
#Initially we con verted the data to datetype
uber_db['request_time']=pd.to_datetime(uber_db['request_time'])
#aggregating total journey time day wise
journey_data=uber_db.groupby('request_hour')['request_time'].sum()
print(journey_data.head())

#Timeseries plot
sns.tsplot(data=journey_data)
plt.show()

#group by month and year to see the highest 
journey_data=uber_db.groupby(['request_month','request_year']).journey_time.mean()

#Combined timeseries data
plt.figure(figsize=(8,6))
sns.tsplot(journey_data)
plt.xlabel("Journey time")
plt.ylabel("request time")
plt.show()



In [ ]:
#pivot data using month as index 
df=pd.pivot_table(uber_db,values='journey_time',index='request_year',columns='request_month',aggfunc='mean')

#creating Heatmap using data
sns.heatmap(df,cmap='Y1GnBu')
#Gives output as how journey time is affected on request month



#Pivot data using month and Nocars available
df=pd.pivot_table(uber_nocars,values='journey_time',index='request_hour',columns='request_day',aggfunc='count')
#Heatmap
sns.heatmap(df,cmap='Y1GnBu')
#Gives Output at what time we got no cars available


#pivot data using month and drivers cancelled
df=pivot_table(uber_null,values='journey_time',index='request_hour',columns='request_month',aggfunc='count')
#Heatmap
sns.heatmap(df,cmap='Y1GnBu')
#This shows at what time driver cancelled at car